# Tutorial 4: Structs and (more on) Types

We will now dig deeper into the idea of multiple dispatch by seeing how "objects" in Julia are defined, constructed, and related to one another. I put "objects" in quotation marks because they are not 100% like objects in the sense of Object Oriented Programming (and, sure, you can do OOP in Julia, although it will be quite awkward). Specifically, Julia objects do not have methods of their own; all methods, even constructors, are defined outside of the object definition block, and their scopes are not surrounded by a bigger scope delimited by the object(s) they work with. 

Before sinking our teeth into the content, I'll clear up some terms (some of which were briefly mentioned in Tutorial 1 but haven't been discussed in detail) and throw in a few tldr notes (as a sneak peek of how Julia is quite different from many popular languages):

- **Types** ≈ objects in OOP languages.
- There are **abstract types** and **concrete types**.
- Types are related to each other by a graph where all non-leaf nodes are abstract types and all leafs are concrete types.
- Concrete types cannot be supertypes of any other type. They are final. 
- Only abstract types are allowed to supertype other types. An abstract type can also can be (and in most cases) is a subtype of another abstract type.
- Concrete types are further divided into primitive types and composite types.
- Composite types are called `struct`s. A `struct` is either immutable (by default) or mutable.


## 1. Structs

Objects in Julia whose types are composite are defined by the `struct` keyword, so we call such objects "structs". A struct consists only of fields that are either primitives or other structs. 

A `struct` block can be as simple as this:

In [15]:
struct Singleton
    x
end

# construct Singleton
a = Singleton('a')
b = Singleton(1)
println("a: ", a, ", b: ", b)

a: Singleton('a'), b: Singleton(1)


Now suppose you want to define a struct that stores the data of data science students in a college. Each field now needs to be statically typed to prevent human errors. If you want to impose such restrictions on the struct's fields, you can declare the type of each field like this:

In [1]:
let
    struct DataScienceStudent
        name::AbstractString
        id::Int
        concentration::AbstractString
        transcript::Dict{AbstractString, AbstractString}
        advisor::AbstractString
        standing::AbstractString
    end

    nigel = DataScienceStudent("Nigel", # name
                               01, # id 
                               "Machine Learning", # concentration
                               Dict("Linear Algebra" => "A"), # transcript
                               "Dr. Alpha Waymond", # advisor
                               "good") # standing
    println("nigel's struct: ", nigel)
end

nigel's struct: DataScienceStudent("Nigel", 1, "Machine Learning", Dict{AbstractString, AbstractString}("Linear Algebra" => "A"), "Dr. Alpha Waymond", "good")


(Note that the code above was wrapped in a `let` block just because we will need to modify the `DataScienceStudent` struct definition later.)

Did you notice that the constructor of each struct above does not need explicit definition? With `DataScienceStudent`, for example, its default constructor is just `DataScienceStudent(<Arguments>)`, in which the arguments must be in the same order as that of the fields according to the struct definition. 

Just like functions, sometimes we want to instantiate a struct using keyword arguments instead of positional ones. Other times, we want the struct to have default values for some of their fields. Multiple dispatch allows us to do so by defining another method for the constructor using keyword arguments and default values. Note that the method should be defined outside of the `struct` block. 

In [2]:
let
    struct SoftwareStudent
        name::AbstractString
        id::Int
        concentration::AbstractString
        transcript::Dict{AbstractString, AbstractString}
        advisor::AbstractString
        standing::AbstractString
    end
    
    # external constructor
    function SoftwareStudent(; name, id, concentration, 
                               transcript=Dict{AbstractString, AbstractString}())
        # name, id, concentration, and transcript are now passed as 
        # keywords args.
        # transcript is assigned to an empty Dict by default.
        # other fields are automatically filled in.
        SoftwareStudent(name, id, concentration, transcript, "Dr. Alpha Evelyn", "good")
    end

    nicole = SoftwareStudent(id=02, name="Nicole", concentration="Game Design")
    println("nicole's struct: ", nicole)
end

nicole's struct: SoftwareStudent("Nicole", 2, "Game Design", Dict{AbstractString, AbstractString}(), "Dr. Alpha Evelyn", "good")


## 2. Abstract types

We've familarized ourselves with concrete types, but how about abstract types? Below are some abstract types defined using the keyword `abstract type`; as you can see, they can be subtypes and supertypes of one another. The symbol `<:` reads "is a subtype of".

In [17]:
abstract type Student end
abstract type ClassOf23 <: Student end # ClassOf23 is a subtype of Student
abstract type CS23 <: ClassOf23 end # CS23 is a subtype of ClassOf23
abstract type Art23 <: ClassOf23 end # Art23 is a subtype of ClassOf23

But what are they good for? You might have guessed something along the line of inheritance. Well, *yesn't*: in Julia, subtypes do not inherit the structures of their supertypes; they only inherit the behaviors defined by methods that accept their supertypes. For example, look at the code below:

In [21]:
# Concrete types that inherit CS23

struct DataScienceStudent <: CS23
    name::AbstractString
    id::Int
    concentration::AbstractString
    transcript::Dict{AbstractString, AbstractString}
    advisor::AbstractString
    standing::AbstractString
end

struct SoftwareStudent <: CS23
    name::AbstractString
    id::Int
    concentration::AbstractString
    transcript::Dict{AbstractString, AbstractString}
    advisor::AbstractString
    standing::AbstractString
end

function SoftwareStudent(; name, id, concentration, 
    transcript=Dict{AbstractString,AbstractString}())
SoftwareStudent(name, id, concentration, transcript, "Dr. Alpha Evelyn", "good")
end

# ==============================================================================

# Concrete type that inherits Art23

struct FineArtStudent <: Art23
    name::AbstractString
    id::Int
    concentration::AbstractString
    transcript::Dict{AbstractString, AbstractString}
    advisor::AbstractString
    standing::AbstractString
end

function FineArtStudent(; name, id, concentration, 
    transcript=Dict{AbstractString,AbstractString}())
FineArtStudent(name, id, concentration, transcript, "Dr. Alpha Gong Gong", "good")
end

# ==============================================================================
# A function whose method is defined for inputs that are subtypes of `Art23`
# and `CS23`:

function dorm_building(; artstudent::Art23, csstudent::CS23)
    if (typeof(artstudent) == FineArtStudent 
     && typeof(csstudent) == DataScienceStudent)
        return "Building A"
    end
    return "Building B"
end

# =============================================================================
# Demo:

nicole = SoftwareStudent(name="Nicole", id=02, concentration="Game Design")
natasha = FineArtStudent(name="Natasha", id=04, concentration="Sculpture")
dorm_building(artstudent=natasha, csstudent=nicole)

"Building B"

Wow, let's slow down and unpack each code block above. First, we define `DataScienceStudent` and `SoftwareStudent` as subtypes of `CS23` (because they both belong to the computer science department of a college). We also defined `FineArtStudent` as a subtype of `Art23`. Then we defined a function called `dorm_building` that does a pretty weird thing: assign a pair of roommates to a dorm building based on their majors. `dorm_building` now only has 1 method that accepts one student from the art department and another from the CS department, which puts all of them in Building B unless one of them studies fine art and the other data science (told you it was weird). Type abstraction allows defining only one method for that task: we do not have to write a dozen other methods to deal with every pair of majors chosen from the art and the CS departments.

You might have noticed that because Julia does not allow structure inheritance, everytime we defined a subtype (namely `DataScienceStudent` and `SoftwareStudent`), we had to copy-paste the fields from this subtype to another. This is restrictive and somewhat annoying, but Julia designers said there were [very good reasons to make it this way](https://docs.julialang.org/en/v1/manual/types/) (although I am not experienced enough to be entirely convinced).

## 3. Parameterization



With Java and many languages that are stingy about memory, you might sometimes come across *parameterization* in the syntactic form of `<Type>{<AnotherType>, <AnotherType>, <AndSoON>}`. Parameterization is one way to strike a balance between being very strict about the type of a field (as in demanding that a field `x` must be `Int64`, for instance) and being very unclear about what types are allowed. It also helps with knowing which method implementation goes with which parameters--think of the differences in implementation and memory between `Array{Bool}`, `Array{String}`, and `Array{DataScienceStudent}`.

To demonstrate the first point, look at the example below. By defining only one parametric type `CrossConcentrationBuddy{Student}`, we have also defined a host of other types like `CrossMajorBuddy{ClassOf23}`
and `CrossMajorBuddy{CS23}`.

In [6]:
let
    abstract type Buddy{Student} end # `Buddy` now takes `Student` as a parameter

    # by declaring CrossConcentrationBuddy{Student}, 
    # CrossConcentrationBuddy{ClassOf23} and CrossConcentrationBuddy{CS23} 
    # can now be declared using the same constructor.
    struct CrossConcentrationBuddy{Student} <: Buddy{Student}
        stuA::Student 
        stuB::Student
    end

    nigel = DataScienceStudent("Nigel",
                               01,
                               "Machine Learning",
                               Dict("Linear Algebra" => "A"),
                               "Dr. Alpha Waymond",
                               "good")

    nicole = SoftwareStudent(id=02, name="Nicole", concentration="Game Design")

    buddies = CrossConcentrationBuddy{ClassOf23}(nigel, nicole) 
    # see the `ClassOf23`?
    
    println(buddies)
end 

CrossConcentrationBuddy{ClassOf23}(DataScienceStudent("Nigel", 1, "Machine Learning", Dict{AbstractString, AbstractString}("Linear Algebra" => "A"), "Dr. Alpha Waymond", "good"), SoftwareStudent("Nicole", 2, "Game Design", Dict{AbstractString, AbstractString}(), "Dr. Alpha Evelyn", "good"))


## 4. Inner constructor

But what we really want to do with `CrossConcentrationBuddy` is to pair 2 students whose concentrations are different. If they are the same, we want to disallow it by throwing an error. In that case, declaring a constructor inside the `struct` block serves to impose that restriction on all constructors that might be defined by multiple dispatch.

In [14]:
let

abstract type Buddy{Student} end
struct CrossConcentrationBuddy{Student} <: Buddy{Student}
    stuA::Student
    stuB::Student

    # inner constructors are used whenever there needs to be a restriction
    # on what arguments are allowed to passed in.
    function CrossConcentrationBuddy{Student}(stuA, stuB)
        if stuA.concentration == stuB.concentration
            throw(ArgumentError("Students must have different concentrations!"))
        end
        new(stuA, stuB)
    end
end

nicole = SoftwareStudent(id=02, name="Nicole", concentration="Game Design")
natalie = SoftwareStudent(id=03, name="Nicole", concentration="Game Design")

# this pair cannot be formed because the 2 students share a concentration
buddies_2 = CrossConcentrationBuddy{Student}(natalie, nicole)
println(buddies_2)

end

ArgumentError: ArgumentError: Students must have different concentrations!

## 5. `Base.show` and other predefined, standard methods

If you're used to OOP, you might be curious of standard methods that come along with all objects. For instance, in Python every object comes with a `__str__()` and a `__repr__()` for free, while in Java it's `toString()`. 

In Julia, the equivalent of such methods is

```
Base.show(io::IO, ::MIME"text/plain", ::Vector{MyType}) = println("new representation")
```
which looks pretty scary. Unfortunately, similar standard methods that are usually taken for granted in Python and Java are not easy to find in Julia (in Python, you just run `dir(x)`, and every method that comes with `x` is shown). And they do not always come from the same module such as `Base`!